In [3]:
import tensorflow as tf
import tensorflow.keras.layers as tfl

In [18]:
class CNNBlock(tfl.Layer):
    def __init__(self, out, **kwargs):
        super().__init__()
        self.relu=tfl.ReLU()
        self.conv=tfl.Conv2D(out,**kwargs)
        self.bn=tfl.BatchNormalization()
    def call(self,x,training=False):
        x = self.conv(x, training = training)
        x = self.bn(x, training=training)
        x = self.relu(x)
        return x

In [36]:
class Inception(tfl.Layer):
    def __init__(self,o_1x1,r_3x3,o_3x3,r_5x5,o_5x5,pool):
        super().__init__()
        self.b1=CNNBlock(o_1x1,kernel_size=1,strides=1)
        self.b2=tf.keras.Sequential([
            CNNBlock(r_3x3,kernel_size=1,strides=1),
            CNNBlock(o_3x3,kernel_size=3,strides=1,padding='same')
        ])
        self.b3=tf.keras.Sequential([
            CNNBlock(r_5x5,kernel_size=1,strides=1),
            CNNBlock(o_5x5,kernel_size=5,strides=1,padding='same')
        ])
        self.b4=tf.keras.Sequential([
            tfl.MaxPooling2D(pool_size=(3,3),strides=1,padding='same'),
            CNNBlock(pool,kernel_size=1,strides=1)
        ])
    def call(self,x,training= False):
        x=tf.concat([
            self.b1(x,training=False),
            self.b2(x,training=False),
            self.b3(x,training=False),
            self.b4(x,training=False)
        ],axis=-1)
        return x

In [31]:
class GoogLeNet(tf.keras.Model):
    def __init__(self,classes=1000):
        super().__init__()
        self.conv1=CNNBlock(64,kernel_size=7,strides=2,padding='same')
        self.maxpool1=tfl.MaxPooling2D(pool_size=(3,3),strides=2,padding='same')
        self.convblock1=tf.keras.Sequential([
            self.conv1,
            self.maxpool1
        ])
        self.convblock2= tf.keras.Sequential([
            CNNBlock(64,kernel_size=1,strides=1,padding='same'),
            CNNBlock(192,kernel_size=3,strides=1,padding='same'),
            tfl.MaxPooling2D(pool_size=(3,3), strides=2, padding='same')
        ])

        self.inception3a = Inception(64,96,128,16,32,32)
        self.inception3b = Inception(128,128,192,32,96,64)
        self.maxpool3 = tfl.MaxPooling2D(pool_size=(3,3),strides=2,padding= 'same')

        self.inception4a = Inception(192,96,208,16,48,64)
        self.inception4b = Inception(160,112,224,24,64,64)
        self.inception4c = Inception(128,128,256,24,64,64)
        self.inception4d = Inception(112,144,288,32,64,64)
        self.inception4e = Inception(256,160,320,32,128,128)
        self.maxpool4 = tfl.MaxPooling2D(pool_size=(3,3),strides=2,padding= 'same')

        self.inception5a = Inception(256, 160, 320, 32, 128, 128)
        self.inception5b = Inception(384, 192, 384, 48, 128, 128)
        
        self.avgpool = tfl.GlobalAveragePooling2D()
        self.dropout = tfl.Dropout(0.4)
        self.dense = tfl.Dense(classes,activation='softmax')

    def call(self,x,training=False):
        x = self.convblock1(x, training=training)
        x = self.convblock2(x, training=training)
        
        x = self.inception3a(x, training=training)
        x = self.inception3b(x, training=training)
        x = self.maxpool3(x)

        x = self.inception4a(x, training=training)
        x = self.inception4b(x, training=training)
        x = self.inception4c(x, training=training)
        x = self.inception4d(x, training=training)
        x = self.inception4e(x, training=training)
        x = self.maxpool4(x)

        x = self.inception5a(x, training=training)
        x = self.inception5b(x, training=training)

        x = self.avgpool(x)
        x = self.dropout(x, training=training)
        x = self.dense(x)
        return x

In [12]:
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.cifar10.load_data()

In [9]:
import numpy as np

In [10]:
x_train.shape

(50000, 32, 32, 3)

In [13]:
def resize_img(image,label):
    image = tf.image.resize(image, (96, 96))
    image = tf.cast(image, tf.float32) / 255.0  
    label = tf.squeeze(label)  
    return image, label

BATCH_SIZE = 64

def create_dataset(x, y):
    return (tf.data.Dataset.from_tensor_slices((x, y))
            .map(resize_img)
            .batch(BATCH_SIZE)
            .prefetch(tf.data.AUTOTUNE))

train_ds = create_dataset(x_train, y_train)
test_ds = create_dataset(x_test, y_test)


NameError: name 'preprocess' is not defined

In [37]:
model = GoogLeNet(classes=10)
dummy_input = tf.random.normal([1, 224, 224, 3])
model(dummy_input)  # Build the model fully
model.summary()


Model: "goog_le_net_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ cnn_block_570 (CNNBlock)             │ (1, 112, 112, 64)           │           9,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_130 (MaxPooling2D)     │ (1, 56, 56, 64)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_290 (Sequential)          │ (1, 56, 56, 64)             │           9,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_291 (Sequential)          │ (1, 28, 28, 192)            │         115,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ inception_90 (Inception)             │ ?                           │         165,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ inception_91 (Inception)             │ ?                           │         391,296 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_134 (MaxPooling2D)     │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ inception_92 (Inception)             │ ?                           │         378,672 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ inception_93 (Inception)             │ ?                           │         451,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ inception_94 (Inception)             │ ?                           │         512,760 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ inception_95 (Inception)             │ ?                           │         608,192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ inception_96 (Inception)             │ ?                           │         872,448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_140 (MaxPooling2D)     │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ inception_97 (Inception)             │ ?                           │       1,047,552 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ inception_98 (Inception)             │ ?                           │       1,449,136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_10          │ ?                           │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (1, 10)                     │          10,250 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,012,922 (22.94 MB)

 Trainable params: 5,998,362 (22.88 MB)

 Non-trainable params: 14,560 (56.88 KB)

In [38]:
model = GoogLeNet(classes=1000)
dummy_input = tf.keras.Input(shape=(224, 224, 3))
output = model(dummy_input)
model_summary = tf.keras.Model(inputs=dummy_input, outputs=output)
model_summary.summary()

Model: "functional_265"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_236 (InputLayer)         │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ goog_le_net_11 (GoogLeNet)           │ (None, 1000)                │       7,027,672 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,027,672 (26.81 MB)

 Trainable params: 7,013,112 (26.75 MB)

 Non-trainable params: 14,560 (56.88 KB)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=20
)

In [ ]:
model.save('Inception.h5')